# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import requests
import json
import os
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice
from azureml.core.webservice import AciWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Read and clean the data

raw_data = pd.read_csv('./data/balanced.csv')
raw_data = raw_data.dropna()
raw_data = raw_data.drop("WO-MRR NUM", axis=1)
raw_data = raw_data.drop("Has MRR", axis=1)
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])
print(raw_data.shape)

(4124, 85)


In [3]:
# Create a new Dataset from the raw_data
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')

dataset = Dataset.Tabular.register_pandas_dataframe(raw_data, datastore ,"Sinter", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/bd9e50d0-5a89-4deb-ad76-018ca1177c15/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [4]:
# choose a name for experiment
experiment_name = 'Sinter_AML'

experiment=Experiment(ws, experiment_name)

# Create a Compute target

In [5]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_settings = {
    "experiment_timeout_minutes": 40,
    "max_concurrent_iterations": 2,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HAS SCRAP AT SINTER",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

Submitting remote run.
No run_configuration provided, running on ML-Cluster with default configuration
Running on remote compute: ML-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Sinter_AML,AutoML_33386fa7-d00f-44bb-a03c-b1933e789deb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and al

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, best_model = remote_run.get_output()

best_model_metrics = best_run.get_metrics()


Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

In [9]:
# Display the metrics of the best model

metrics = pd.DataFrame.from_dict(best_model_metrics, orient='index', columns=['metric value'])
metrics

,metric value
balanced_accuracy,0.64
f1_score_macro,0.64
precision_score_macro,0.64
matthews_correlation,0.29
f1_score_weighted,0.64
norm_macro_recall,0.29
AUC_weighted,0.69
average_precision_score_weighted,0.67
average_precision_score_micro,0.67
average_precision_score_macro,0.67


In [10]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162658/code/Users/odl_user_162658'}

prefittedsoftvotingclassifier
{'estimators': ['13', '7', '12', '16', '20', '0', '11', '31'],
 'weights': [0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]}

13 - maxabsscaler
{'copy': True}

13 - sgdclassifierwrapper
{'alpha': 7.5510448979591835,
 'class_weight': 'balanced',
 'eta0': 0.001,
 'fit_intercept': True,
 'l1_ratio': 0.42857142857142855,
 'learning_rate': 'constant',
 'loss': 'modified_huber',
 'max_iter': 1000,
 'n_jobs': 1,
 'penalty': 'none',
 'power_t': 0.7777777777777777,
 'random_state': None,
 'tol': 0.0001}

7 - maxabsscaler
{'copy': True}

7 - logisticregression
{'C': 1.

In [10]:
#TODO: Save the best model

aml_model = remote_run.register_model(model_name='AML_Best', tags=best_model_metrics)



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
# Define inference config

env = best_run.get_environment()
inference_config = InferenceConfig(
    environment= env,
    source_directory = "./source_dir",
    entry_script="score.py"
)

In [26]:
# Define a deployment configuration on the Webservice

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True, enable_app_insights=True
)

In [27]:
# Deploy the model

service = Model.deploy(
    workspace = ws,
    name = "sinter-scrap-model1",
    models = [aml_model],
    inference_config = inference_config,
    deployment_config = deploy_config,
    overwrite = True
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-20 17:05:29+00:00 Creating Container Registry if not exists.
2021-10-20 17:05:29+00:00 Registering the environment.
2021-10-20 17:05:29+00:00 Use the existing image.
2021-10-20 17:05:29+00:00 Generating deployment configuration.
2021-10-20 17:05:30+00:00 Submitting deployment to compute.
2021-10-20 17:05:33+00:00 Checking the status of deployment sinter-scrap-model1..
2021-10-20 17:10:57+00:00 Checking the status of inference endpoint sinter-scrap-model1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(service.get_logs())

2021-10-20T16:41:41,966028900+00:00 - rsyslog/run 
2021-10-20T16:41:41,963577600+00:00 - nginx/run 
2021-10-20T16:41:41,984021800+00:00 - iot-server/run 
2021-10-20T16:41:41,997521800+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_1b22a9c5d9b447e8fbdcc34171e02fe8/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-20T16:41:42,537530000+00:00 - iot-server/finish 1 0
2021-10-20T16:41:42,539346700+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (65)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 89
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-10-20 16:41:45,561 | root | INFO | Starting up app insights client
logging socket was 

In [15]:
print(service.scoring_uri)

http://b1d21690-1514-4d1b-9656-0a1cc00c9f6a.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
import json 
import numpy as np

test_data = raw_data[0:10]
test_data = test_data.drop("HAS SCRAP AT SINTER", axis=1)
test_dict = ({'data':test_data.to_dict(orient='records')})
test_sample = json.dumps(test_dict)

#json_data = json.loads(sample)['data']

#pd_data = pd.DataFrame(json_data)

#print(pd_data.shape)

#np_array = np.array(json_data)

#len(np_array)

print(service.run(test_sample))

[0, 1, 1, 1, 1, 1, 0, 1, 0, 1]


In [43]:
service.get_logs()

'2021-10-19T13:03:18,233826700+00:00 - nginx/run \n2021-10-19T13:03:18,245304000+00:00 - iot-server/run \n2021-10-19T13:03:18,314432800+00:00 - rsyslog/run \n2021-10-19T13:03:18,313497200+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\nrsyslogd: /azureml-envs/azureml_1b22a9c5d9b447e8fbdcc34171e02fe8/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-10-19T13:03:19,027141200+00:00 - iot-server/finish 1 0\n2021-10-19T13:03:19,037201000+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (64)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 89\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-10-19 13:03:22,706 | root | INFO | Starting up app insights client\

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
